In [0]:
#get darknet repo from github

!git clone https://github.com/AlexeyAB/darknet


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [0]:
#build darknet
!make

In [0]:
#download pretrained weights

!wget https://pjreddie.com/media/files/yolov3.weights


In [0]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path)

In [0]:
#test darknet detection

!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg


In [0]:
imShow('predictions.jpg')

In [0]:
#run on custom data
%cd darknet
!./darknet detect cfg/yolov3.cfg yolov3.weights  ../fruits.png


In [0]:
#show result
imShow('predictions.jpg')

TRAINING CUSTOM OBJECT DETECTION


In [0]:
#create custom dataset
#do it on computer

!git clone https://github.com/theAIGuysCode/OIDv4_ToolKit.git


In [0]:
pip install -r requirements.txt

In [0]:
!python OIDv4_ToolKit/main.py downloader --classes Knife Spoon Spatula Measuring_cup Frying_pan --type_csv train --limit 150 --multiclasses 1

In [0]:
!python convert_annotations.py
#do it on computer till here


In [2]:
cd darknet


/content/darknet


In [0]:
#in darknet folder
!gsutil -q -m cp /content/drive/My\ Drive/YOLOv3_Kitchen_Utensils/obj.zip ./



In [0]:
#to it in darknet folder
!unzip ./obj.zip -d data/

In [0]:
!gsutil -q -m cp ../darknet/cfg/yolov3.cfg  /content/drive/My\ Drive/YOLOv3_Kitchen_Utensils/yolov3_custom.cfg


#note to future me:
#To copy files from colab to drive use this answer from stackoverflow:
#https://stackoverflow.com/questions/56578685/how-to-copy-files-from-colab-or-cloud-storage-to-google-drive

#problems I faced
#1. CP command did not work ,use gsutil instead
# you need to change directory to drive

#Make changes to custom config file in an editor.
# comment batch=1 and subdivisions
# uncomment batch and subdivisions 
#change num of classes in Yolo 
#change num of filters in convolutional part above yolo 
#save and do next!




In [0]:
#do it in darknet folder
!gsutil -q -m cp ../drive/My\ Drive/YOLOv3_Kitchen_Utensils/yolov3_custom.cfg  ./cfg 



In [0]:
#do it in darknet folder

!gsutil -q -m cp ../drive/My\ Drive/YOLOv3_Kitchen_Utensils/obj.names  data
!gsutil -q -m cp ../drive/My\ Drive/YOLOv3_Kitchen_Utensils/obj.data  data




In [0]:
#do it in darknet folder
!gsutil -q -m cp ../drive/My\ Drive/YOLOv3_Kitchen_Utensils/generate_train.py ./



In [0]:
#do it in darknet folder

!python generate_train.py


In [0]:
!ls data/
#output sould contain train.txt file

In [11]:
# upload pretrained convolutional layer weights

!wget http://pjreddie.com/media/files/darknet53.conv.74

URL transformed to HTTPS due to an HSTS policy
--2020-05-18 12:27:20--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   197KB/s    in 17m 7s  

2020-05-18 12:44:29 (154 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [0]:
cd darknet/


In [0]:
!ln -s /content/drive/My\ Drive/  /mydrive


In [0]:
#TRAINING CUSTOM OBJECT DETECT

!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

In [0]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show

In [0]:
imShow('chart.png')

NameError: ignored

In [0]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

In [0]:
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights /mydrive/images/safari.jpg -thresh 0.3
imShow('predictions.jpg')


!gsutil -q -m cp ../drive/My\ Drive/YOLOv3_Kitchen_Utensils/yolov3_custom.cfg  ./cfg 
